# Imports and paths

In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import glob
import torch.optim as optim
from tqdm import tqdm
import sys

# Paths for each view (update these paths with the correct file locations)
train_axial_path = 'train/axial/'
valid_axial_path = 'valid/axial/'
train_coronal_path = 'train/coronal/'
valid_coronal_path = 'valid/coronal/'
train_sagittal_path = 'train/sagittal/'
valid_sagittal_path = 'valid/sagittal/'

# CSV paths for labels (update these paths with the correct file locations)
train_labels_abnormal = 'train-abnormal.csv'
valid_labels_abnormal = 'valid-abnormal.csv'
train_labels_acl = 'train-acl.csv'
valid_labels_acl = 'valid-acl.csv'
train_labels_meniscus = 'train-meniscus.csv'
valid_labels_meniscus = 'valid-meniscus.csv'

# Class for my MRIDataset

In [14]:
class MRIDataset(Dataset):
    def __init__(self, axial_path, coronal_path, sagittal_path, abnormal_labels_path, acl_labels_path, meniscus_labels_path, transform=None):
        # Collecting all file paths for each orientation into one list
        self.all_images = sorted(glob.glob(os.path.join(axial_path, '*.npy')) +
                                 glob.glob(os.path.join(coronal_path, '*.npy')) +
                                 glob.glob(os.path.join(sagittal_path, '*.npy')))
        self.transform = transform

        # Load labels
        abnormal_labels = pd.read_csv(abnormal_labels_path, header=None)[1].to_numpy()
        acl_labels = pd.read_csv(acl_labels_path, header=None)[1].to_numpy()
        meniscus_labels = pd.read_csv(meniscus_labels_path, header=None)[1].to_numpy()

        # Stack labels to create a (num_samples, 3) array (1 label per sample)
        self.labels = np.vstack((abnormal_labels, acl_labels, meniscus_labels)).T

    def __len__(self):
        # The length of the dataset is the total number of images (across all orientations)
        return len(self.all_images)

    def __getitem__(self, idx):
        # Load the image from the unified list of all images
        image = np.load(self.all_images[idx])

        # Take the middle slice if images have 3 dimensions
        def middle_slice(image):
            return image[image.shape[0] // 2] if image.ndim == 3 else image

        image = middle_slice(image)

        # Convert the image to PIL format and apply transformations if any
        image = Image.fromarray((image * 255).astype(np.uint8))  # Rescale to [0, 255] for display
        if self.transform:
            image = self.transform(image)  # Apply any transform passed

        # For labels, we return the label corresponding to the current index
        label = torch.tensor(self.labels[idx % len(self.labels)], dtype=torch.float32)

        return image, label

# Preprocessing and augmentation transforms

In [15]:
preprocess_transforms = transforms.Compose([
    transforms.Resize((128, 128)),                   # Resize to 128x128
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])      # Normalize with mean=0.5, std=0.5 (assuming grayscale)
])

# Additional augmentation transforms
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=10),           # Randomly rotate image by up to 10 degrees
    transforms.RandomHorizontalFlip(),               # Randomly flip horizontally
    preprocess_transforms                            # Apply preprocessing after augmentations
])

# Train dataset

In [16]:
# Initialize Dataset with augmentation transforms for training
train_dataset = MRIDataset(
    axial_path=train_axial_path, coronal_path=train_coronal_path, sagittal_path=train_sagittal_path,
    abnormal_labels_path=train_labels_abnormal, acl_labels_path=train_labels_acl, meniscus_labels_path=train_labels_meniscus,
    transform=augmentation_transforms
)

# Initialize DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# Valid Dataset

In [17]:
valid_dataset = MRIDataset(
    axial_path=valid_axial_path, coronal_path=valid_coronal_path, sagittal_path=valid_sagittal_path,
    abnormal_labels_path=valid_labels_abnormal, acl_labels_path=valid_labels_acl, meniscus_labels_path=valid_labels_meniscus,
    transform=preprocess_transforms
)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# My simple convolutional neural network

In [18]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, 3)  # 3 output classes for multi-label classification
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Training and validation loop

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 1e-3
num_epochs = 10
# Initialize model, loss function, and optimizer
model = SimpleCNN()
model.to(device)  # Move model to GPU if available
criterion = nn.BCEWithLogitsLoss()  # Use binary cross-entropy for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
def train_epoch(model, dataloader, optimizer, criterion, epoch):
    model.train()
    total_train_loss = 0.0
    dataset_size = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader), colour='cyan', file=sys.stdout)
    for step, (images, labels) in bar:
        # We take the images and their labels and push them on GPU
        images = images.to(device)
        labels = labels.to(device)

        batch_size = images.shape[0]

        # Reset gradients
        optimizer.zero_grad()

        # Obtain predictions
        pred = model(images)

        # Compute loss for this batch
        loss = criterion(pred, labels)

        # Compute gradients for each weight (backpropagation)
        loss.backward()

        # Update weights based on gradients (gradient descent)
        optimizer.step()

        # We keep track of the average training loss
        total_train_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = np.round(total_train_loss / dataset_size, 2)
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss)

    return epoch_loss

In [21]:
def valid_epoch(model, dataloader, criterion, epoch):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    dataset_size = 0
    correct = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader), colour='cyan', file=sys.stdout)

    for step, (images, labels) in bar:  # Expect two values: images and labels
        images = images.to(device)
        labels = labels.to(device).float()  # Ensure labels are floats for BCEWithLogitsLoss

        batch_size = images.shape[0]

        pred = model(images)  # Predictions should have shape [batch_size, num_classes]
        loss = criterion(pred, labels)  # Ensure labels have same shape as pred

        # For evaluation: Convert predictions to binary (0/1) using threshold
        predictions = (torch.sigmoid(pred) > 0.5).int()

        # Calculate the number of correct predictions
        correct += (predictions == labels.int()).sum().item()

        # Compute loss and accuracy
        total_val_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = np.round(total_val_loss / dataset_size, 2)
        accuracy = np.round(100 * correct / (dataset_size * labels.size(1)), 2)  # Normalize by total number of elements

        bar.set_postfix(Epoch=epoch, Valid_Acc=accuracy, Valid_Loss=epoch_loss)

    return accuracy, epoch_loss


In [22]:
def run_training(model, criterion, trainloader, testloader, optimizer, num_epochs):
    # Check if we are using GPU
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    # For keeping track of the best validation accuracy
    top_accuracy = 0.0

    # We train the model for a number of epochs
    for epoch in range(num_epochs):
        # Train the model for one epoch
        train_loss = train_epoch(model, trainloader, optimizer, criterion, epoch)

        # For validation we do not keep track of gradients
        with torch.no_grad():
            val_accuracy, val_loss = valid_epoch(model, testloader, criterion, epoch)
            # Save the best model based on validation accuracy
            if val_accuracy > top_accuracy:
                print(f"Validation Accuracy Improved ({top_accuracy} ---> {val_accuracy})")
                top_accuracy = val_accuracy

        # Print a new line after each epoch
        print()

run_training(model, criterion, train_loader, valid_loader, optimizer, num_epochs)

[INFO] Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU

100%|██████████| 12/12 [00:00<00:00, 16.97it/s, Epoch=0, Valid_Acc=63.4, Valid_Loss=0.75]
Validation Accuracy Improved (0.0 ---> 63.43)

100%|██████████| 12/12 [00:00<00:00, 26.63it/s, Epoch=1, Valid_Acc=65.7, Valid_Loss=0.63]
Validation Accuracy Improved (63.43 ---> 65.65)

100%|██████████| 12/12 [00:00<00:00, 51.05it/s, Epoch=2, Valid_Acc=66.3, Valid_Loss=0.63]
Validation Accuracy Improved (65.65 ---> 66.3)

100%|██████████| 12/12 [00:00<00:00, 55.37it/s, Epoch=3, Valid_Acc=67.1, Valid_Loss=0.62]
Validation Accuracy Improved (66.3 ---> 67.13)

100%|██████████| 12/12 [00:00<00:00, 53.23it/s, Epoch=4, Valid_Acc=67, Valid_Loss=0.61] 

100%|██████████| 12/12 [00:00<00:00, 54.69it/s, Epoch=5, Valid_Acc=67.9, Valid_Loss=0.64]
Validation Accuracy Improved (67.13 ---> 67.87)

100%|██████████| 12/12 [00:00<00:00, 52.26it/s, Epoch=6, Valid_Acc=69.4, Valid_Loss=0.62]
Validation Accuracy Improved (67.87 ---> 69.44)

100%|██████████| 12/12 [00

# Testing and Evaluation

In [23]:
def evaluate_model(model, dataloader):
    model.eval()
    all_labels, all_outputs = [], []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            all_labels.append(labels.cpu())
            all_outputs.append(outputs.cpu())
    
    all_labels = torch.cat(all_labels).numpy()
    all_outputs = torch.cat(all_outputs).numpy()
    all_outputs = 1 / (1 + np.exp(-all_outputs))  # Apply sigmoid

    # Convert outputs to binary predictions
    predictions = (all_outputs > 0.5)

    # Calculate multi-label accuracy
    correct = (predictions == all_labels).sum()
    total = np.prod(all_labels.shape)  # Total number of elements
    accuracy = correct / total

    f1 = f1_score(all_labels, predictions, average='macro')
    roc_auc = roc_auc_score(all_labels, all_outputs, average='macro')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")


# Run evaluation

In [24]:
evaluate_model(model, valid_loader)

Accuracy: 0.7000
F1 Score: 0.5776
ROC AUC: 0.7383
